In [10]:
model = "featurestore"

In [2]:
from hops import featurestore
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from hops import experiment
from tensorflow.python.keras.callbacks import TensorBoard
from hops import tensorboard

In [3]:
NUM_EPOCHS = 30
BATCH_SIZE = 10
LEARNING_RATE = 0.001
SHUFFLE_BUFFER_SIZE = 10000
INPUT_SHAPE=4

In [4]:
def create_tf_dataset():
    dataset_dir = featurestore.get_training_dataset_path("predict_house_sold_for_dataset")
    input_files = tf.gfile.Glob(dataset_dir + "/part-r-*")
    dataset = tf.data.TFRecordDataset(input_files)
    tf_record_schema = featurestore.get_training_dataset_tf_record_schema("predict_house_sold_for_dataset")
    feature_names = ["avg_house_age", "avg_house_size", "avg_house_worth", "avg_num_bidders"]
    label_name = "avg_sold_for"
    def decode(example_proto):
        example = tf.parse_single_example(example_proto, tf_record_schema)
        x = []
        for feature_name in feature_names:
            x.append(example[feature_name])
        y = [tf.cast(example[label_name], tf.float32)]
        return x,y
    dataset = dataset.map(decode).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat(NUM_EPOCHS)
    return dataset

In [5]:
def create_model():
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_shape = (INPUT_SHAPE,), batch_size=BATCH_SIZE),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)])
    return model

In [6]:
def export_model(classifier):
    """
    Exports trained model 
    
    Args:
        :classifier: the model to export
    """
    def _serving_input_receiver_fn():
        # key (e.g. 'examples') should be same with the inputKey when you 
        # buid the request for prediction
        receiver_tensors = {"dense_input":tf.placeholder(dtype=tf.float32,shape=[1,INPUT_SHAPE])}
        return tf.estimator.export.ServingInputReceiver(receiver_tensors, receiver_tensors)
    from hops import serving
    from hops import hdfs
    export_path = hdfs.project_path()+"/Models/" + model                                                      
    if not hdfs.exists(export_path):
        hdfs.mkdir(export_path)
    classifier.export_savedmodel(export_path, _serving_input_receiver_fn)
#     serving.export(os.getcwd() + "/model","featurestore", 2

In [7]:
def train_fn():
    
    import tensorflow as tf
    import numpy as np
    from hops import tensorboard
    from hops import hdfs
    
    model = create_model()
    model.compile(optimizer=tf.train.AdamOptimizer(LEARNING_RATE), loss='mse',  metrics=['accuracy'])
    run_config = tf.contrib.learn.RunConfig(
        model_dir=tensorboard.logdir(),
        save_checkpoints_steps=10,
        save_summary_steps=5,
        log_step_count_steps=10)
    keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, 
               config=run_config, model_dir=tensorboard.logdir())
    tf.estimator.train_and_evaluate(keras_estimator, train_spec=tf.estimator.TrainSpec(input_fn=create_tf_dataset),
        eval_spec=tf.estimator.EvalSpec(input_fn=create_tf_dataset))
    export_model(keras_estimator)

In [8]:
experiment.launch(train_fn, 
                  name="feature store quick start example", 
                  local_logdir=True)

Finished Experiment 

'hdfs://10.0.2.15:8020/Projects/demo_deep_learning_admin000/Experiments/application_1561468620886_0012/launcher/run.1'